In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import f1_score, classification_report
import numpy as np

2024-02-05 17:40:25.200597: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 17:40:25.304376: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-05 17:40:25.309585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-05 17:40:25.309607: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
data_g1 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G1.xlsx")
data_g2 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G2.xlsx")
data_g3 = pd.read_excel("/home/ubuntu/training/ayush/continual_learning/G3.xlsx")

data_g1.shape, data_g2.shape, data_g3.shape

data_g1.drop('Unnamed: 0', axis =1, inplace = True)
data_g2.drop('Unnamed: 0', axis =1, inplace = True)
data_g3.drop('Unnamed: 0', axis =1, inplace = True)

In [3]:
data_g2['text'] = data_g2['text'].astype(str)

data_g1['tags'] = data_g1['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))
data_g2['tags'] = data_g2['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))
data_g3['tags'] = data_g3['tags'].apply(lambda x: re.sub(r"[^a-zA-Z_]", " ", x))

data_g1['tags'] = data_g1['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))
data_g2['tags'] = data_g2['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))
data_g3['tags'] = data_g3['tags'].apply(lambda x: ', '.join(word.strip() for word in set(x.split())))

In [4]:
data_g1_train, data_g1_test = train_test_split(data_g1, test_size = 0.2, random_state = 42)
data_g2_train, data_g2_test = train_test_split(data_g2, test_size = 0.2, random_state = 42)
data_g3_train, data_g3_test = train_test_split(data_g3, test_size = 0.2, random_state = 42)
data_g1_train.shape, data_g1_test.shape, data_g2_train.shape, data_g2_test.shape, data_g3_train.shape, data_g3_test.shape

((5884, 3), (1472, 3), (5164, 3), (1291, 3), (5021, 3), (1256, 3))

In [5]:
def preprocess_data(data):
    max_words = 10000 
    max_length = 300  
 
    tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(data['text'])
 
    sequences = tokenizer.texts_to_sequences(data['text'])
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')
 
    mlb = MultiLabelBinarizer()
    encoded_tags = mlb.fit_transform(data['tags'].apply(lambda x: set(x.split(', '))))
 
    return {'text': padded_sequences, 'tags': encoded_tags}

In [6]:
def create_and_train_model(data, model_name, vocab_size, embedding_dim, max_length, num_classes):
    model = keras.Sequential([
        layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(data['text'], data['tags'], epochs=50)

    model.save(f'/home/ubuntu/training/ayush/continual_learning/models/{model_name}.h5')

    return model

In [7]:
data1_train = preprocess_data(data_g1_train)
data1_test = preprocess_data(data_g1_test)

data2_train = preprocess_data(data_g2_train)
data2_test = preprocess_data(data_g2_test)

data3_train = preprocess_data(data_g3_train)
data3_test = preprocess_data(data_g3_test)

In [8]:
data_combined = {
    'text': np.concatenate([data1_train['text'], data2_train['text'], data3_train['text']]),
    'tags': np.concatenate([data1_train['tags'], data2_train['tags'], data3_train['tags']])
}

## Training model on combined data

In [9]:
model_comb = create_and_train_model(data_combined, 'model_combined', vocab_size = 10000, embedding_dim = 50, max_length = 300, num_classes = 4)

Epoch 1/50


2024-02-05 17:41:14.831105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-05 17:41:14.831129: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-05 17:41:14.831146: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-0-71): /proc/driver/nvidia/version does not exist
2024-02-05 17:41:14.831424: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


503/503 [==============================] - 2s 3ms/step - loss: 0.3978 - accuracy: 0.6051
Epoch 2/50
503/503 [==============================] - 1s 3ms/step - loss: 0.2420 - accuracy: 0.7858
Epoch 3/50
503/503 [==============================] - 1s 3ms/step - loss: 0.1502 - accuracy: 0.8643
Epoch 4/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0864 - accuracy: 0.8983
Epoch 5/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.9147
Epoch 6/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0255 - accuracy: 0.9207
Epoch 7/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0149 - accuracy: 0.9228
Epoch 8/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0102 - accuracy: 0.9253
Epoch 9/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0076 - accuracy: 0.9269
Epoch 10/50
503/503 [==============================] - 1s 3ms/step - loss: 0.0064 - accuracy: 0.9245
Epoch 11/5

## performance on combined data

In [10]:
data_test_combined = {
    'text': np.concatenate([data1_test['text'], data2_test['text'], data3_test['text']]),
    'tags': np.concatenate([data1_test['tags'], data2_test['tags'], data3_test['tags']])
}

In [11]:
model_comb = keras.models.load_model("/home/ubuntu/training/ayush/continual_learning/models/model_combined.h5")

In [12]:
tags_order = ['allergy_name', 'cancer', 'chronic_disease', 'treatment']

def evaluate_model(model, test_data, tags_order):
    predictions = model.predict(test_data['text'])

    binary_predictions = (predictions > 0.5).astype(int)

    true_labels = test_data['tags']

    f1_scores = []
    for i in range(len(tags_order)):
        tag_f1 = f1_score(true_labels[:, i], binary_predictions[:, i])
        print(f"F1 score for tag '{tags_order[i]}': {tag_f1}")
        f1_scores.append(tag_f1)
        
    print("\nOverall Classification Report:\n", classification_report(true_labels, binary_predictions, target_names=tags_order))

    return f1_scores

f1_scores_data1 = evaluate_model(model_comb, data_test_combined, tags_order)

126/126 [==============================] - 0s 1ms/step
F1 score for tag 'allergy_name': 0.05952380952380952
F1 score for tag 'cancer': 0.2279245283018868
F1 score for tag 'chronic_disease': 0.4484278577725536
F1 score for tag 'treatment': 0.5328269126424309

Overall Classification Report:
                  precision    recall  f1-score   support

   allergy_name       0.10      0.04      0.06       116
         cancer       0.24      0.22      0.23       684
chronic_disease       0.50      0.41      0.45      1883
      treatment       0.55      0.52      0.53      1885

      micro avg       0.47      0.42      0.44      4568
      macro avg       0.34      0.30      0.32      4568
   weighted avg       0.47      0.42      0.44      4568
    samples avg       0.47      0.42      0.44      4568



/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
